This script is meant to run in a ROS1 environment to convert the lidar data in the livox_ros_driver message type to livox_ros_driver2 message type

The livox_ros_driver is at
https://github.com/Livox-SDK/livox_ros_driver

The livox_ros_driver2 is at
https://github.com/Livox-SDK/livox_ros_driver2


Once the bag file is converted to livox_ros_driver2 in ROS 1, it can be quickly converted to ROS 2 by the rosbags-convert command. The rosbags package can be installed from pip
```
pip3 install rosbags
```

To convert to ROS 2, we need the definition of the message in ROS 2, which can be downloaded at:
https://github.com/brytsknguyen/livox_ros_driver2


In [ ]:
import os, sys, glob, time

import numpy as np
import pandas as pd

import math

# Parallel proces
from myparproc.ParProc import ParProc
from myparproc.ProgressCheck import ProgressCheck

# Linear algebra
import myutil.myutil as util

# Plotting tools
import matplotlib.pyplot as plt


import rospy, rosbag

# For logging to csv
import csv

from livox_ros_driver.msg import CustomMsg as lvCloud
from livox_ros_driver.msg import CustomPoint as lvPoint

from livox_ros_driver2.msg import CustomMsg as lvCloud2
from livox_ros_driver2.msg import CustomPoint as lvPoint2

# For kd tree search
# import pcl, pcl_ros, pcl_msgs

# For load/unloading pcd/ply
# import pypcd
# from pypcd import pypcd

# For loading/unloading pcd/ply
# from pyntcloud import PyntCloud

# Some processing or plotting
# import open3d as o3d

# from ceva import Ceva

# Lidar topic
lidar_topic = '/livox/lidar'

# Location of the data in livox_ros_driver
dataset_loc = '/media/tmn/mySataSSD2/MCDVIRAL_slictros2/ntu_day_01/ntu_day_01_mid70.bag'

# Output in livox_ros_driver2
output_loc = '/media/tmn/mySataSSD2/MCDVIRAL_slictros2/ntu_day_01/ntu_day_01_mid70_driver2'

# Minimum distance
min_range = 0.75

# Extrinsic of livox lidar wrt to body frame
atv_T_B_Ll = np.array([[ 0.99985813160508060000, -0.0002258196228773494, -0.0168423771687589430, -0.010514839241742317],
                       [-0.00017111407692153792, -0.9999947058517530000,  0.0032494597148797887, -0.008989784841758377],
                       [-0.01684302179448474500, -0.0032461167514232880, -0.9998528768488223000,  0.037356468638334630],
                       [ 0.00000000000000000000,  0.0000000000000000000,  0.0000000000000000000,  1.000000000000000000]])

hhs_T_B_Ll = np.array([[ 1.0,  0.0,  0.0, 0.020],
                       [ 0.0, -1.0,  0.0, 0.000],
                       [ 0.0,  0.0, -1.0, 0.037],
                       [ 0.0,  0.0,  0.0, 1.000]])

# Extrinsic of ouster lidar wrt to body frame
atv_T_B_Lo = np.array([[ 0.999934655205122900,  0.003477624535771754, -0.010889970036688295, -0.060649229060416594],
                       [ 0.003587143302461965, -0.999943027982117100,  0.010053516443599904, -0.012837544242408117],
                       [-0.010854387257665576, -0.010091923381711220, -0.999890161647627000, -0.020492606896077407],
                       [ 0.000000000000000000,  0.000000000000000000,  0.000000000000000000,  0.000000000000000000]])

hhs_T_B_Lo = np.array([[ 0.999913504074183700, -0.011166365511073898, -0.006949579221822984, -0.048945211204946950],
                       [-0.011356389542502144, -0.999545300686582400, -0.027932495268565650, -0.031269290603480840],
                       [-0.006634514801117132,  0.028009001350326540, -0.999585653686922000, -0.017555157942225650],
                       [ 0.000000000000000000,  0.000000000000000000,  0.000000000000000000,  0.000000000000000000]])
# Merge rate
step_size = 200

# Number of points from ouster to be downsampled to
os_ds_points = 50000


In [ ]:
# Check for existing livox and ouster data
bag_files = glob.glob(dataset_loc)
os.makedirs(output_loc, exist_ok=True)

for bag in bag_files:

    # Get the sequence name
    seq = os.path.basename(bag).replace('.bag', '')

    # Read the bag file
    livox_ros_driver1_bag = rosbag.Bag(bag, 'r')

    # Create the output the bag file
    livox_ros_driver2_bag = rosbag.Bag(output_loc + '/' + seq + '_driver2_.bag', 'w')

    # Read the livox messages into buffer
    bag_info = livox_ros_driver1_bag.get_type_and_topic_info()
    total_messages = sum([info.message_count for topic, info in bag_info.topics.items()])
    lvpc = []
    prock = ProgressCheck(total_messages)
    for topic, msg, t in livox_ros_driver1_bag.read_messages():
        out_msg = lvCloud2()
        out_msg.header    = msg.header
        out_msg.timebase  = msg.timebase
        out_msg.point_num = msg.point_num
        out_msg.lidar_id  = msg.lidar_id
        out_msg.rsvd      = msg.rsvd
        out_msg.points    = msg.points
        print(f"time: {msg.points[0].offset_time:d}, {msg.points[-1].offset_time:d}.")
        livox_ros_driver2_bag.write('/livox/lidar', out_msg, t)
        prock.updateProgress(f"Coverting cloud {seq} ")

    livox_ros_driver1_bag.close()
    livox_ros_driver2_bag.close()